import re

from langchain.agents import AgentExecutor, create_react_agent, Tool
from langchain_community.tools import ArxivQueryRun, DuckDuckGoSearchRun
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama 
from langchain_community.document_loaders import WebBaseLoader

## Papers Finder Agent

In [43]:
def create_finder_agent(llm):
    """
    Creates a research agent equipped with Arxiv and DuckDuckGo search tools.

    Args:
        llm: An instance of a LangChain language model.

    Returns:
        An AgentExecutor ready to process research queries.
    """
    # --- 2. Define the Tools ---
    # The agent will have access to two tools:
    # - Arxiv: For searching academic papers specifically on arxiv.org.
    # - DuckDuckGoSearch: For general web searches to find papers on other sites
    #   or to get a broader context.
    arxiv_tool = ArxivQueryRun()
    ddg_search_tool = DuckDuckGoSearchRun()

    tools = [
        Tool(
            name="arxiv_search",
            func=arxiv_tool.run,
            description="Use this tool to search for research papers on arXiv.org. The input should be a search query."
        ),
        Tool(
            name="web_search",
            func=ddg_search_tool.run,
            description="Use this tool for general web searches to find research papers or articles not available on arXiv."
        ),
    ]

    # --- 3. Create the Agent's Prompt ---
    # This prompt acts as the agent's "brain," instructing it on its role,
    # how to use its tools, and how to format its final answer.
    # The ReAct (Reason+Act) framework is used here, which is excellent for
    # tool-using agents.
    prompt_template = """
    You are a diligent and expert research assistant. Your goal is to find relevant research papers on a given topic.
    To answer the user's request, you must follow these steps:
    1.  Start by using the `arxiv_search` tool to see if you can find papers on the primary repository.
    2.  If the arXiv search is not sufficient or you need broader context, use the `web_search` tool.
    3.  Analyze the results from your tools.
    4.  When you have found enough information and are confident in your findings, provide a final answer.
    5.  Your final answer should be a formatted list of the papers you found, including their titles and links.

    You have access to the following tools:
    {tools}

    Use the following format for your reasoning process:

    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of [{tool_names}]
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    Thought: I now know the final answer
    Final Answer: the final answer to the original input question, which should be a formatted list of papers. Each paper should be separated by a 
    unique string like 'Paper:'

    Begin!

    Question: {input}
    Thought: {agent_scratchpad}
    """

    prompt = ChatPromptTemplate.from_template(prompt_template)

    # --- 4. Create the Agent ---
    # We use the create_react_agent function, which creates an agent that
    # uses the ReAct framework to decide which tool to use based on the
    # user's input and its previous actions.
    agent = create_react_agent(llm, tools, prompt)

    # --- 5. Create the Agent Executor ---
    # The AgentExecutor is the runtime for the agent. It's what actually
    # calls the agent, executes the chosen tools, and passes the results
    # back to the agent for the next reasoning step.
    agent_executor = AgentExecutor(
        agent=agent,
        tools=tools,
        verbose=True,  # Set to True to see the agent's thought process
        handle_parsing_errors=True  # Helps with robustness
    )

    return agent_executor

In [20]:
llm = ChatOllama(model="gemma3:12b", temperature=0)

In [21]:
finder_agent = create_finder_agent(llm)

In [22]:
topic = "multi-agent systems with orchestrators"

In [ ]:
response = finder_agent.invoke({"input": topic})

In [24]:
print("\n--- Final Answer ---")
print(response["output"])


--- Final Answer ---
Paper: Intelligent Data-Driven Architectural Features Orchestration for Network Slicing
[https://arxiv.org/abs/2401.11799](https://arxiv.org/abs/2401.11799)

Paper: Joint$\lambda$: Orchestrating Serverless Workflows on Jointcloud FaaS Systems
[https://arxiv.org/abs/2405.15379](https://arxiv.org/abs/2405.15379)

Paper: Live Orchestral Piano, a system for real-time orchestral music generation
[https://arxiv.org/abs/1705.05779](https://arxiv.org/abs/1705.05779)


## Summarizer Agent

In [38]:
def get_webpage_content(url: str) -> str:
    """
    Fetches the text content of a webpage given its URL.
    It cleans up whitespace and truncates the content to a manageable size for the LLM.
    """
    try:
        loader = WebBaseLoader(url)
        docs = loader.load()
        content = " ".join([doc.page_content for doc in docs])
        # Clean up excessive whitespace and limit length to avoid overwhelming the context window
        cleaned_content = " ".join(content.split())
        max_chars = 15000
        return cleaned_content[:max_chars]
    except Exception as e:
        return f"Error fetching content from {url}: {e}"

In [39]:
def create_summarizer_agent(llm):
    """
    Creates a summarization agent that can read a webpage and summarize it.
    """
    web_fetch_tool = Tool(
        name="web_content_fetcher",
        func=get_webpage_content,
        description="Use this tool to fetch the text content of a webpage given its URL. The input must be a single URL."
    )

    tools = [web_fetch_tool]

    prompt_template = """
    You are an expert academic summarizer. Your goal is to provide a concise summary of a research paper.

    You have access to the following tool:
    {tools}

    The user will provide the paper's title and a URL. Follow these steps:
    1. Use the `web_content_fetcher` tool with the provided URL to get the paper's text content.
    2. Read the content and create a summary that includes the main objective, methodology, and key findings of the paper.
    3. Present the summary clearly.

    Use the following format:

    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of [{tool_names}]
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    Thought: I now know the final answer
    Final Answer: A summary of the paper.

    Begin!

    Question: {input}
    Thought: {agent_scratchpad}
    """

    prompt = ChatPromptTemplate.from_template(prompt_template)
    agent = create_react_agent(llm, tools, prompt)
    agent_executor = AgentExecutor(
        agent=agent,
        tools=tools,
        verbose=True,
        handle_parsing_errors=True
    )
    return agent_executor

In [40]:
# Create and run the summarizer agent (imported from the new file)
summarizer_agent = create_summarizer_agent(llm)

In [ ]:
for paper in response['output'].split('Paper:')[1:]:
    
    paper_title = paper.split('\n')[0] 
    paper_url = paper.split('\n')[1]
    
    # Format the input for the summarizer agent
    summarizer_input = f"Summarize the paper titled '{paper_title}' from the URL: {paper_url}"

    summary_response = summarizer_agent.invoke({"input": summarizer_input})
    
    print(summary_response)

## Orchestrator

In [61]:
def create_orchestrator_agent(llm):
    """
    Creates the master orchestrator agent that can delegate tasks to the
    researcher and summarizer helper_agents.
    """
    # 1. Instantiate the sub-helper_agents (which will become tools)
    research_agent_executor = create_finder_agent(llm)
    summarizer_agent_executor = create_summarizer_agent(llm)

    # 2. Create tools for the orchestrator to use
    tools = [
        Tool(
            name="research_paper_finder",
            # MODIFICATION: Wrap invoke to return only the output string
            func=lambda query: research_agent_executor.invoke({"input": query})["output"],
            description="""
            Use this tool to find research papers on a specific academic topic.
            The input should be a clear, concise research topic (e.g., 'quantum computing advancements').
            This tool will return a list of relevant papers with their titles and URLs.
            """
        ),
        Tool(
            name="paper_summarizer",
            # MODIFICATION: Wrap invoke to return only the output string
            func=lambda query: summarizer_agent_executor.invoke({"input": query})["output"],
            description="""
            Use this tool to summarize a specific research paper.
            The input must be a string containing the paper's title and its URL,
            like this: 'Summarize the paper titled "Paper Title" from the URL: http://example.com/paper.pdf'
            """
        )
    ]

    # 3. Create the prompt for the orchestrator
    prompt_template = """
    You are an expert orchestrator agent. Your job is to understand a user's request and delegate tasks to specialized helper_agents.
    You have two helper_agents at your disposal: a 'research_paper_finder' and a 'paper_summarizer'.

    - If the user wants to find papers, use the `research_paper_finder`.
    - If the user wants to summarize a paper, use the `paper_summarizer`.
    - If the user asks for a multi-step task (e.g., find and then summarize), you must perform the steps sequentially.
      First, call the `research_paper_finder`, then use its output to call the `paper_summarizer`.

    You have access to the following tools:
    {tools}

    Use the following format:

    Question: the input question you must answer
    Thought: you should always think about what to do. This is your reasoning step.
    Action: the action to take, should be one of [{tool_names}]
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    Thought: I now know the final answer
    Final Answer: the final answer to the original input question

    Begin!

    Question: {input}
    Thought: {agent_scratchpad}
    """

    prompt = ChatPromptTemplate.from_template(prompt_template)

    # 4. Create the orchestrator agent
    orchestrator_agent = create_react_agent(llm, tools, prompt)

    # 5. Create the agent executor
    agent_executor = AgentExecutor(
        agent=orchestrator_agent,
        tools=tools,
        verbose=True,
        handle_parsing_errors=True
    )

    return agent_executor

In [62]:
llm = ChatOllama(model="gemma3:12b", temperature=0)

In [63]:
orchestrator = create_orchestrator_agent(llm)

In [66]:
task = """
    First, find research papers about 'multi-agent ai systems with orchestrators'.
    Then, take the first paper you find and summarize it.
    """

In [ ]:
response = orchestrator.invoke({"input": task})

In [ ]:
print("\n--- Orchestrator Final Answer ---")
print(response["output"])